# <span style="color:green;">SCRAPEANDO EN STEAM:</span>

In [2]:
import requests
url_base = "https://store.steampowered.com/"

In [3]:
from bs4 import BeautifulSoup

In [4]:
import re
respuesta_articulo=requests.get('https://store.steampowered.com/')
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido =sopa_articulo.find_all("div", {"class":"button_container"})
print(contenido)
# de la pagina principal, sacamos las cuatro ventanas pero aun no estan limpias:
# noveades, ofertas , juegos gratuitos y por etiqueta.

[<div class="button_container" data-panel='{"flow-children":"row"}'>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/explore/new/?snr=1_4_4__146">
                    New Releases                </a>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/search/?specials=1&amp;snr=1_4_4__146">
                    Specials                </a>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/genre/Free to Play/?snr=1_4_4__146">
                    Free Games                </a>
<a class="big_button" data-panel="[]" href="https://store.steampowered.com/tag/browse/?snr=1_4_4__146">
                    By User Tags                </a>
</div>]


In [5]:
import re
x = re.findall('https[\S ]+6',str(contenido))
print(x) # obtenemos los links de interes 

['https://store.steampowered.com/explore/new/?snr=1_4_4__146', 'https://store.steampowered.com/search/?specials=1&amp;snr=1_4_4__146', 'https://store.steampowered.com/genre/Free to Play/?snr=1_4_4__146', 'https://store.steampowered.com/tag/browse/?snr=1_4_4__146']


## <span style="color:red;">1) Scrapeando novedades en Steam:</span>

In [6]:
def links(link): 
    return x[link]
links(0) # obtenemos el link de novedades

'https://store.steampowered.com/explore/new/?snr=1_4_4__146'

In [21]:
respuesta_articulo=requests.get(links(0))
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido2 =sopa_articulo.find_all("div", {"class":"tab_see_more"})
#"class":"tab_item_name" 
x2 = re.findall('https[\S]+pop.+n',str(contenido2)) 
print(x2)
# obtenemos el link para ver una lista de todas las novedades posibles.

['https://store.steampowered.com/search/?filter=popularnew&amp;sort_by=Released_DESC&amp;os=win']


### <span style="color:blue;">Nombres</span>

In [9]:
# nombres
def nombres(cantidad):
    respuesta_articulo=requests.get(x2[0])
    sopa_articulo = BeautifulSoup(respuesta_articulo.text)
    contenido3 =sopa_articulo.find_all("div", {"class":"col search_name ellipsis"})
    lista=[]
    for i in range(cantidad):
        lista.append(contenido3[i].text)
    lista2=[]
    for word in lista:
        lista2.append(word.replace("\n",""))  
    return lista2
NOMBRES=nombres(50) # la cantidad mostrada varia de acuerdo al usuario
            # po l o que se establece un rango minimo de 50.

### <span style="color:blue;">Fecha de lanzamiento</span>

In [11]:
def fecha(cantidad):
    lista=[]
    for i in range(cantidad):
        respuesta_articulo=requests.get(x2[0])
        sopa_articulo = BeautifulSoup(respuesta_articulo.text)
        contenido5 =sopa_articulo.find_all("div", {"col search_released responsive_secondrow"})
        lista.append(contenido5[i].text)
    return lista
FECHA=fecha(50)

### <span style="color:blue;">Reseñas de los usuarios (Valoración, cantidad de reseñas y calificación final)</span>

In [12]:
respuesta_articulo=requests.get(x2[0])
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido7 =sopa_articulo.find_all("span", {"class":"search_review_summary"})
x3 = re.findall('[0-9]+[\S ]+ive.',str(contenido7)) 
valoracion=re.findall('[0-9]+%',str(x3))
reseñas=re.findall(' [0-9,]+',str(x3))
resumen=re.findall('[MVPO][A-z ]+',str(contenido7))
# se extrae 3 caracteristicas importantes que se encuentran dentro del a reseña

### <span style="color:blue;">Precio</span>

In [119]:
#Obtención de los precios finales de los juegos
respuesta_articulo=requests.get(x2[0]) # Sacamos el request de la página de novedades totales
sopa_articulo = BeautifulSoup(respuesta_articulo.text) # Lo guardamos en la variable sopa articulo el texto obtenido
contenido6 =sopa_articulo.find_all("div", {"class":"responsive_search_name_combined"}) # Guardamos las filas de los juegos
import re # Importamos re para hacer uso de las expresiones regulares
x6 = re.findall('data-price-.+',str(contenido6)) # Con expresiones regulares sacamos nuestra variable objetivo
PRECIO = re.findall('[0-9]+',str(x6)) # Obtenemos unicamente los caracteres númericos

### <span style="color:blue;">Creando el dataframe de lo obtenido</span>

In [14]:
import pandas as pd
df = pd.DataFrame()
df["Nombre"]=NOMBRES
df["Fecha de lanzamiento"]=FECHA
df["Valoración"]=valoracion
df["Cantidad de reseñas"]=reseñas
df["Calificación final"]=resumen
df["Precio"]=PRECIO

In [15]:
df['Precio'] = df['Precio'].astype(float, errors = 'raise') # hacemos la tranformación a flotante.
precioN=df["Precio"]/100 
df["Precio"]=precioN
df

,Nombres,Fecha,Valoración,Cantidad de Reseñas,Calificación Final,Precio
0,ELDEN RING,"24 Feb, 2022",87%,"253,836",Very Positive,199.00
1,Counter-Strike: Global Offensive,"21 Aug, 2012",88%,"6,294,159",Very Positive,58.00
2,Lost Ark,"11 Feb, 2022",74%,"142,315",Mostly Positive,0.00
3,Apex Legends™,"4 Nov, 2020",86%,"418,217",Very Positive,0.00
4,MONSTER HUNTER RISE,"12 Jan, 2022",87%,"16,163",Very Positive,131.93
5,It Takes Two,"25 Mar, 2021",96%,"68,754",Overwhelmingly Positive,55.60
6,Destiny 2,"1 Oct, 2019",84%,"454,816",Very Positive,0.00
7,Tom Clancy's Rainbow Six® Siege,"1 Dec, 2015",87%,"876,258",Very Positive,23.99
8,NARAKA: BLADEPOINT,"11 Aug, 2021",80%,"84,558",Very Positive,59.00
9,PUBG: BATTLEGROUNDS,"21 Dec, 2017",55%,"1,985,219",Mixed,0.00


## <span style="color:red;">2) Scrapeando juegos en promociones en Steam:</span>

In [22]:
def links(link): 
    return x[link]
prom = links(1) # obtenemos el link de juegos en promociones
prom

'https://store.steampowered.com/search/?specials=1&amp;snr=1_4_4__146'

### <span style="color:blue;">Nombres</span>

In [46]:
# nombres
def nombres(cantidad):
    respuesta_articulo=requests.get(prom)
    sopa_articulo = BeautifulSoup(respuesta_articulo.text)
    contenido3 =sopa_articulo.find_all("div", {"class":"col search_name ellipsis"})
    lista=[]
    for i in range(cantidad):
        lista.append(contenido3[i].text)
    lista2=[]
    for word in lista:
        lista2.append(word.replace("\n",""))  
    return lista2
NOMBRES=nombres(50) # la cantidad mostrada varia de acuerdo al usuario
            # po l o que se establece un rango minimo de 50.

### <span style="color:blue;">Fecha de lanzamiento</span>

In [51]:
def fecha(cantidad):
    lista=[]
    for i in range(cantidad):
        respuesta_articulo=requests.get(prom)
        sopa_articulo = BeautifulSoup(respuesta_articulo.text)
        contenido5 =sopa_articulo.find_all("div", {"col search_released responsive_secondrow"})
        lista.append(contenido5[i].text)
    return lista
FECHA=fecha(50)

### <span style="color:blue;">Reseñas de los usuarios (Valoración, cantidad de reseñas y calificación final)</span>

In [47]:
respuesta_articulo=requests.get(prom)
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido7 =sopa_articulo.find_all("span", {"class":"search_review_summary"})
x3 = re.findall('[0-9]+[\S ]+ive.',str(contenido7)) 
valoracion=re.findall('[0-9]+%',str(x3))
print(valoracion)
reseñas=re.findall(' [0-9,]+',str(x3))
print(reseñas)
resumen=re.findall('[MVPO][A-z ]+',str(contenido7))
print(resumen)
# se extrae 3 caracteristicas importantes que se encuentran dentro del a reseña

['96%', '86%', '87%', '87%', '80%', '87%', '83%', '88%', '86%', '85%', '53%', '81%', '90%', '95%', '97%', '71%', '90%', '86%', '81%', '54%', '84%', '86%', '95%', '94%', '83%', '98%', '87%', '89%', '86%', '80%', '82%', '75%', '93%', '74%', '90%', '81%', '98%', '85%', '67%', '85%', '82%', '78%', '94%', '84%', '85%', '94%', '86%', '97%', '87%', '86%']
[' 68,769', ' 43,035', ' 876,272', ' 16,165', ' 43,842', ' 229,272', ' 147,807', ' 12,348', ' 209,529', ' 73,832', ' 14,971', ' 419,389', ' 73,696', ' 15,014', ' 59,013', ' 86,082', ' 73,696', ' 20,467', ' 156,551', ' 14,458', ' 223,987', ' 71,647', ' 48,669', ' 44,315', ' 62,854', ' 106,004', ' 16,205', ' 105,691', ' 7,864', ' 52,929', ' 30,334', ' 10,782', ' 24,523', ' 57,337', ' 7,120', ' 155,856', ' 31,845', ' 23,765', ' 67,353', ' 7,388', ' 30,952', ' 35,231', ' 87,224', ' 32,339', ' 57,311', ' 4,294', ' 750', ' 15,883', ' 18,455', ' 70,081']
['Overwhelmingly Positive', 'Very Positive', 'Very Positive', 'Very Positive', 'Very Positive',

### <span style="color:blue;">Precio inicial</span>

In [48]:
respuesta_articulo=requests.get(prom)
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido5 =sopa_articulo.find_all("div", {"class":"col search_price discounted responsive_secondrow"})
x7 = re.findall('strike>S..[0-9.]+',str(contenido5))
PRECIOI = re.findall('S..[0-9.]+',str(x7))

### <span style="color:blue;">Precio final</span>

In [49]:
respuesta_articulo=requests.get(prom)
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido5 =sopa_articulo.find_all("div", {"class":"col search_price discounted responsive_secondrow"})
PRECIOF = re.findall('S..[0-9]+.[0-9]+ ',str(contenido5))

### <span style="color:blue;">Descuento</span>

In [50]:
import re
respuesta_articulo=requests.get(prom)
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido6 =sopa_articulo.find_all("span")
descuento = re.findall('-[0-9]+%',str(contenido6)) # porcentaje de descuento

### <span style="color:blue;">Creando el dataframe de lo obtenido</span>

In [45]:
import pandas as pd
df = pd.DataFrame()
df["Nombre"]=NOMBRES
df["Fecha de lanzamiento"]=FECHA
df["Valoración"]=valoracion
df["Cantidad de reseñas"]=reseñas
df["Calificación final"]=resumen
df["Precio inicial"]=PRECIOI
df["Precio final"]=PRECIOF
df["Descuento"]=descuento
df

,Nombre,Fecha de lanzamiento,Valoración,Cantidad de reseñas,Calificación final,Precio inicial,Precio final,Descuento
0,It Takes Two,"25 Mar, 2021",96%,"68,754",Overwhelmingly Positive,S/.139.00,S/.55.60,-60%
1,The Sims™ 4,"18 Jun, 2020",86%,"43,032",Very Positive,S/.139.00,S/.16.68,-88%
2,Tom Clancy's Rainbow Six® Siege,"1 Dec, 2015",87%,"876,258",Very Positive,S/.59.99,S/.23.99,-60%
3,MONSTER HUNTER RISE,"12 Jan, 2022",87%,"16,163",Very Positive,S/.199.90,S/.131.93,-34%
4,FIFA 22,"30 Sep, 2021",80%,"43,823",Very Positive,S/.199.00,S/.65.67,-67%
5,Red Dead Redemption 2,"5 Dec, 2019",87%,"229,260",Very Positive,S/.199.90,S/.99.95,-50%
6,Sid Meier’s Civilization® VI,"20 Oct, 2016",83%,"147,800",Very Positive,S/.239.00,S/.35.85,-85%
7,NieR Replicant™ ver.1.22474487139...,"23 Apr, 2021",88%,"12,347",Very Positive,S/.199.00,S/.99.50,-50%
8,Monster Hunter: World,"9 Aug, 2018",86%,"209,525",Very Positive,S/.57.28,S/.28.64,-50%
9,NieR:Automata™,"17 Mar, 2017",85%,"73,831",Very Positive,S/.159.00,S/.79.50,-50%


## <span style="color:red;">3) Scrapeando los juegos más vendidos en Steam:</span>

In [64]:
import re
respuesta_articulo=requests.get('https://store.steampowered.com/')
sopa_articulo = BeautifulSoup(respuesta_articulo.text)
contenido =sopa_articulo.find_all("div", {"class":"tab_see_more"})
print(contenido[1])
# de la pagina principal, obtenemos la vista completa de los juegos más vendidos

<div class="tab_see_more">
							See more:							<a class="btnv6_white_transparent btn_small_tall" href="https://store.steampowered.com/search/?filter=topsellers&amp;os=win&amp;snr=1_4_4__tabsmore">
<span>Top Sellers</span>
</a>
</div>


In [65]:
import re
top = re.findall('https[\S ]+re',str(contenido[1]))
print(top[0]) # obtenemos unicamente el link de los juegos más vendidos de steam

https://store.steampowered.com/search/?filter=topsellers&amp;os=win&amp;snr=1_4_4__tabsmore


### <span style="color:blue;">Nombres</span>

In [73]:
# nombres
def nombres(cantidad):
    respuesta_articulo=requests.get(top[0])
    sopa_articulo = BeautifulSoup(respuesta_articulo.text)
    contenido3 =sopa_articulo.find_all("div", {"class":"col search_name ellipsis"})
    lista=[]
    for i in range(cantidad):
        lista.append(contenido3[i].text)
    lista2=[]
    for word in lista:
        lista2.append(word.replace("\n",""))  
    return lista2
NOMBRES=nombres(50) # la cantidad mostrada varia de acuerdo al usuario
            # po l o que se establece un rango minimo de 50.

### <span style="color:blue;">Fecha de lanzamiento</span>

In [75]:
def fecha(cantidad):
    lista=[]
    for i in range(cantidad):
        respuesta_articulo=requests.get(top[0])
        sopa_articulo = BeautifulSoup(respuesta_articulo.text)
        contenido5 =sopa_articulo.find_all("div", {"col search_released responsive_secondrow"})
        lista.append(contenido5[i].text)
    return lista
FECHA=fecha(50)

### <span style="color:blue;">Precio</span>

In [120]:
#Obtención de los precios finales de los juegos
respuesta_articulo=requests.get(top[0]) # Sacamos el request de la página de novedades totales
sopa_articulo = BeautifulSoup(respuesta_articulo.text) # Lo guardamos en la variable sopa articulo el texto obtenido
contenido6 =sopa_articulo.find_all("div", {"class":"responsive_search_name_combined"}) # Guardamos las filas de los juegos
import re # Importamos re para hacer uso de las expresiones regulares
x6 = re.findall('data-price-.+',str(contenido6)) # Con expresiones regulares sacamos nuestra variable objetivo
PRECIO = re.findall('[0-9]+',str(x6)) # Obtenemos unicamente los caracteres númericos

### <span style="color:blue;">Creando el dataframe de lo obtenido</span>

In [114]:
import pandas as pd
df = pd.DataFrame()
df["Nombre"]=NOMBRES
df["Fecha de lanzamiento"]=FECHA
df["Precio"]=PRECIO

In [121]:
df['Precio'] = df['Precio'].astype(float, errors = 'raise') # hacemos la tranformación a flotante.
precioN=df["Precio"]/100 
df["Precio"]=precioN
df

,Nombre,Fecha de lanzamiento,Precio
0,ELDEN RING,"24 Feb, 2022",1.9900
1,It Takes Two,"25 Mar, 2021",0.5560
2,MONSTER HUNTER RISE,"12 Jan, 2022",1.3193
3,Core Keeper,"8 Mar, 2022",0.2800
4,Dread Hunger,"26 Jan, 2022",0.5300
5,NieR Replicant™ ver.1.22474487139...,"23 Apr, 2021",0.9950
6,EA Play,"31 Aug, 2020",0.1900
7,Grand Theft Auto V: Premium Edition,,0.4917
8,NieR:Automata™,"17 Mar, 2017",0.7950
9,Red Dead Redemption 2,"5 Dec, 2019",0.9995
